In [1]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score, classification_report, f1_score, accuracy_score
from nltk.metrics import agreement

## Data Loading and Cleaning

In [2]:
social_gold_df = pd.read_csv('social_toy_corpus_FULL_v2.csv', encoding='utf8')
nurse_gold_df = pd.read_csv('nurse_toy_corpus_FULL.csv', encoding='utf8')
phys_gold_df = pd.read_csv('physician_toy_corpus_FULL.csv', encoding='utf8')
social_gold_df = social_gold_df[['text','Race_group', 'insurance', 'Gender']]
nurse_gold_df = nurse_gold_df[['text','Race_group', 'insurance', 'Gender']]
phys_gold_df = phys_gold_df[['text','Race_group', 'insurance', 'Gender']]
social_gold_df.loc[(social_gold_df['Race_group']!='White_NonHispanic', 'Race_group')] = 'NonWhite'
nurse_gold_df.loc[(nurse_gold_df['Race_group']!='White_NonHispanic', 'Race_group')] = 'NonWhite'
phys_gold_df.loc[(phys_gold_df['Race_group']!='White_NonHispanic', 'Race_group')] = 'NonWhite'
social_gold_df.loc[(social_gold_df['Race_group']!='NonWhite', 'Race_group')] = 'White'
nurse_gold_df.loc[(nurse_gold_df['Race_group']!='NonWhite', 'Race_group')] = 'White'
phys_gold_df.loc[(phys_gold_df['Race_group']!='NonWhite', 'Race_group')] = 'White'

In [3]:
mg_social = pd.read_csv('./annotated_data/social_toy_corpus_MG_v2.csv', encoding='utf8')
mg_nurse = pd.read_csv('./annotated_data/nurse_toy_corpus_MG.csv', encoding='utf8')
mg_phys = pd.read_csv('./annotated_data/physician_toy_corpus_MG.csv', encoding='utf8')
db_social = pd.read_csv('./annotated_data/social_toy_corpus_DB_v2.csv', encoding='utf8')
db_nurse = pd.read_csv('./annotated_data/nurse_toy_corpus_DB.csv', encoding='utf8')
db_phys = pd.read_csv('./annotated_data/physician_toy_corpus_DB.csv', encoding='utf8')
ad_social = pd.read_csv('./annotated_data/social_toy_corpus_AD_v2.csv', encoding='utf8')
ad_nurse = pd.read_csv('./annotated_data/nurse_toy_corpus_AD.csv', encoding='utf8')
ad_phys = pd.read_csv('./annotated_data/physician_toy_corpus_AD.csv', encoding='utf8')

In [4]:
sc_social = pd.read_csv('./annotated_data/shan_data/social_toy_corpus_SC_v2.csv', encoding='utf8')
sc_nurse = pd.read_csv('./annotated_data/shan_data/nurse_toy_corpus_SC.csv', encoding='utf8')
sc_phys = pd.read_csv('./annotated_data/shan_data/physician_toy_corpus_SC.csv', encoding='utf8')

In [5]:
sm_social = pd.read_excel('/home/marco/Desktop/nethome/SDOH/annotation/race_ins_gen/annotated_data/social_toy_corpus_SM_v2.xlsx', sheet_name='social_toy_corpus_SC_v2')
sm_nurse = pd.read_excel('/home/marco/Desktop/nethome/SDOH/annotation/race_ins_gen/annotated_data/nurse_toy_corpus_SMEdits.xlsm', sheet_name='nurse_toy_corpus_SC_SMEdits')
sm_phys = pd.read_csv('/home/marco/Desktop/nethome/SDOH/annotation/race_ins_gen/annotated_data/physician_toy_corpus_SMEdits.csv')

In [6]:
sm_social['Race'].fillna('White', inplace=True)
sm_social['Insurance'].fillna('reg', inplace=True)
sm_social['Gender'].fillna('Female', inplace=True)
sm_social['Race'] = sm_social['Race'].str.strip()
sm_social['Race'] = sm_social['Race'].str.capitalize()
sm_social['Race'] = sm_social['Race'].replace('Non hispanic white', 'White')
sm_social.loc[(sm_social['Race']!='White', 'Race')] = 'NonWhite'
sm_social['Insurance'] = sm_social['Insurance'].replace('Regular', 'reg')
sm_social['Insurance'] = sm_social['Insurance'].replace('Low income ', 'low')
sm_social['Gender'] = sm_social['Gender'].str.strip()
sm_social['Gender'] = sm_social['Gender'].str.capitalize()
sm_social['Gender'] = sm_social['Gender'].replace('F', 'Female')
sm_social['Gender'] = sm_social['Gender'].replace('Femlae', 'Female')

In [7]:
sm_nurse['Race'].fillna('White', inplace=True)
sm_nurse['Insurance'].fillna('reg', inplace=True)
sm_nurse['Gender'].fillna('Female', inplace=True)
sm_nurse['Race'] = sm_nurse['Race'].str.strip()
sm_nurse['Race'] = sm_nurse['Race'].str.capitalize()
sm_nurse.loc[(sm_nurse['Race']!='White', 'Race')] = 'NonWhite'
sm_nurse['Insurance'] = sm_nurse['Insurance'].str.strip()
sm_nurse['Insurance'] = sm_nurse['Insurance'].replace('regular', 'reg')
sm_nurse['Gender'] = sm_nurse['Gender'].str.strip()
sm_nurse['Gender'] = sm_nurse['Gender'].str.capitalize()
sm_nurse['Gender'] = sm_nurse['Gender'].replace('Man', 'Male')
sm_nurse['Gender'] = sm_nurse['Gender'].replace('Woman', 'Female')

In [8]:
sm_phys['Race'].fillna('White', inplace=True)
sm_phys['Insurance'].fillna('reg', inplace=True)
sm_phys['Gender'].fillna('Female', inplace=True)
sm_phys['Race'] = sm_phys['Race'].str.strip()
sm_phys['Race'] = sm_phys['Race'].str.capitalize()
sm_phys.loc[(sm_phys['Race']!='White', 'Race')] = 'NonWhite'
sm_phys['Insurance'] = sm_phys['Insurance'].str.strip()
sm_phys['Insurance'] = sm_phys['Insurance'].replace('regular', 'reg')
sm_phys['Gender'] = sm_phys['Gender'].str.strip()
sm_phys['Gender'] = sm_phys['Gender'].str.capitalize()
sm_phys['Gender'] = sm_phys['Gender'].replace('Man', 'Male')
sm_phys['Gender'] = sm_phys['Gender'].replace('Woman', 'Female')

In [9]:
sc_phys.fillna('unk', inplace=True)
sc_phys['Race'] = sc_phys['Race'].replace('unk', 'White')
sc_phys['Race'] = sc_phys['Race'].str.capitalize()
sc_phys.loc[(sc_phys['Race']!='White', 'Race')] = 'NonWhite'
sc_phys['Insurance'] = sc_phys['Insurance'].replace('unk', 'reg')
sc_phys['Insurance'] = sc_phys['Insurance'].replace('regular ', 'reg')
sc_phys['Insurance'] = sc_phys['Insurance'].replace('regular', 'reg')
sc_phys['Gender'] = sc_phys['Gender'].replace('unk', 'Female')
sc_phys['Gender']= sc_phys['Gender'].str.capitalize()
sc_phys.loc[47, 'Gender'] = 'Female'

In [10]:
sc_nurse['race'] = sc_nurse['race'].replace('unk', 'White')
sc_nurse['race'] = sc_nurse['race'].str.capitalize()
sc_nurse.loc[(sc_nurse['race']!='White', 'race')] = 'NonWhite'
sc_nurse['insurance '] = sc_nurse['insurance '].replace('unk', 'reg')
sc_nurse['insurance '] = sc_nurse['insurance '].replace('regular', 'reg')
sc_nurse['insurance '] = sc_nurse['insurance '].replace('regular ', 'reg')
sc_nurse['gender'] = sc_nurse['gender'].replace('unk', 'Female')
sc_nurse['gender'] = sc_nurse['gender'].replace(' male', 'Male')
sc_nurse['gender']= sc_nurse['gender'].str.capitalize()
sc_nurse.columns = sc_nurse.columns.str.capitalize()
sc_nurse.columns = sc_nurse.columns.str.strip()

In [11]:
sc_social['Race'] = sc_social['Race'].replace('unk', 'White')
sc_social['Race'] = sc_social['Race'].str.capitalize()
sc_social.loc[(sc_social['Race']!='White', 'Race')] = 'NonWhite'
sc_social['Insurance'] = sc_social['Insurance'].replace('unk', 'reg')
sc_social['Insurance'] = sc_social['Insurance'].replace('regular', 'reg')
sc_social['Insurance'] = sc_social['Insurance'].replace('regular ', 'reg')
sc_social['Gender'] = sc_social['Gender'].replace('unk', 'Female')
sc_social['Gender']= sc_social['Gender'].str.capitalize()
sc_social['Gender'] = sc_social['Gender'].replace('Famle', 'Female')

In [12]:
ad_social.loc[(ad_social['Race']!='White', 'Race')] = 'NonWhite'
ad_nurse.loc[(ad_nurse['Race']!='White', 'Race')] = 'NonWhite'
ad_phys.loc[(ad_phys['Race']!='White', 'Race')] = 'NonWhite'

In [13]:
db_social.loc[(db_social['Race']!='White', 'Race')] = 'NonWhite'
db_nurse.loc[(db_nurse['Race']!='White', 'Race')] = 'NonWhite'
db_phys.loc[(db_phys['Race']!='White', 'Race')] = 'NonWhite'

In [14]:
ad_social.loc[(ad_social['Insurance']!='Low', 'Insurance')] = 'reg'
ad_nurse.loc[(ad_nurse['Insurance']!='Low', 'Insurance')] = 'reg'
ad_phys.loc[(ad_phys['Insurance']!='Low', 'Insurance')] = 'reg'
ad_social.loc[(ad_social['Insurance']!='reg', 'Insurance')] = 'low'
ad_nurse.loc[(ad_nurse['Insurance']!='reg', 'Insurance')] = 'low'
ad_phys.loc[(ad_phys['Insurance']!='reg', 'Insurance')] = 'low'
ad_social.loc[42,'Gender'] = 'Female'

In [15]:
db_social['Insurance'] = db_social['Insurance'].replace('Low', 'low')
db_social.loc[(db_social['Insurance']!='low', 'Insurance')] = 'reg'
db_nurse.loc[(db_nurse['Insurance']!='low', 'Insurance')] = 'reg'
db_phys.loc[(db_phys['Insurance']!='low', 'Insurance')] = 'reg'

In [16]:
db_social['Gender'] = db_social['Gender'].replace('F', 'Female')
db_social['Gender'] = db_social['Gender'].replace('M', 'Male')
db_nurse['Gender'] = db_nurse['Gender'].replace('F', 'Female')
db_nurse['Gender'] = db_nurse['Gender'].replace('M', 'Male')
db_phys['Gender'] = db_phys['Gender'].replace('F', 'Female')
db_phys['Gender'] = db_phys['Gender'].replace('M', 'Male')

## Social Work Notes

#### Race

In [17]:
print('Kappa Scores Annotators:')
print('AD v MG:', cohen_kappa_score(ad_social['Race'], mg_social['Race']))
print('AD v DB:',cohen_kappa_score(ad_social['Race'], db_social['Race']))
print('DB v MG',cohen_kappa_score(db_social['Race'], mg_social['Race']))
print('SC v MG',cohen_kappa_score(sc_social['Race'], mg_social['Race']))
print('SC v DB',cohen_kappa_score(sc_social['Race'], db_social['Race']))
print('SC v AD',cohen_kappa_score(sc_social['Race'], ad_social['Race']))
print('SM v MG',cohen_kappa_score(sm_social['Race'], mg_social['Race']))
print('SM v DB',cohen_kappa_score(sm_social['Race'], db_social['Race']))
print('SM v AD',cohen_kappa_score(sm_social['Race'], ad_social['Race']))
print('SM v SC',cohen_kappa_score(sm_social['Race'], sc_social['Race']))

Kappa Scores Annotators:
AD v MG: 0.6770025839793281
AD v DB: 0.7178329571106095
DB v MG 0.6287128712871287
SC v MG 0.25595238095238093
SC v DB 0.13366336633663367
SC v AD 0.289405684754522
SM v MG 0.289405684754522
SM v DB 0.37923250564334077
SM v AD 0.18414918414918413
SM v SC 0.16020671834625322


In [18]:
print('Kappa Scores Gold:')
print('MG',cohen_kappa_score(social_gold_df['Race_group'], mg_social['Race']))
print('AD',cohen_kappa_score(social_gold_df['Race_group'], ad_social['Race']))
print('DB',cohen_kappa_score(social_gold_df['Race_group'], db_social['Race']))
print('SC',cohen_kappa_score(social_gold_df['Race_group'], sc_social['Race']))
print('SM',cohen_kappa_score(social_gold_df['Race_group'], sm_social['Race']))

Kappa Scores Gold:
MG 0.25742574257425743
AD 0.2663656884875846
DB 0.45175438596491224
SC -0.11386138613861396
SM 0.37923250564334077


In [19]:
print('Accuracy Scores')
print('MG',accuracy_score(social_gold_df['Race_group'], mg_social['Race']))
print('AD',accuracy_score(social_gold_df['Race_group'], ad_social['Race']))
print('DB',accuracy_score(social_gold_df['Race_group'], db_social['Race']))
print('SC',accuracy_score(social_gold_df['Race_group'], sc_social['Race']))
print('SM',accuracy_score(social_gold_df['Race_group'], sm_social['Race']))

Accuracy Scores
MG 0.76
AD 0.74
DB 0.8
SC 0.64
SM 0.78


In [20]:
print('F1 minority class as positive')
print('MG', f1_score(social_gold_df['Race_group'], mg_social['Race'], pos_label='NonWhite'))
print('AD', f1_score(social_gold_df['Race_group'], ad_social['Race'], pos_label='NonWhite'))
print('DB', f1_score(social_gold_df['Race_group'], db_social['Race'], pos_label='NonWhite'))
print('SC', f1_score(social_gold_df['Race_group'], sc_social['Race'], pos_label='NonWhite'))
print('SM', f1_score(social_gold_df['Race_group'], sm_social['Race'], pos_label='NonWhite'))

F1 minority class as positive
MG 0.4
AD 0.43478260869565216
DB 0.5833333333333334
SC 0.1
SM 0.5217391304347826


In [21]:
fleiss_data = [('a1',i, mg_social.loc[i, 'Race'] ) for i in range(len(mg_social))]
fleiss_data.extend([('a2',i, ad_social.loc[i, 'Race'] ) for i in range(len(ad_social))])
fleiss_data.extend([('a3',i, db_social.loc[i, 'Race'] ) for i in range(len(db_social))])
fleiss_data.extend([('a4',i, sc_social.loc[i, 'Race'] ) for i in range(len(sc_social))])
fleiss_data.extend([('a5',i, sm_social.loc[i, 'Race'] ) for i in range(len(sm_social))])
sr_task = agreement.AnnotationTask(data=fleiss_data)
print('Fleiss Kappa:',sr_task.multi_kappa())

Fleiss Kappa: 0.37609183928125806


#### Gender

In [22]:
print('Kappa Scores Annotators:')
print('AD v MG:', cohen_kappa_score(ad_social['Gender'], mg_social['Gender']))
print('AD v DB:',cohen_kappa_score(ad_social['Gender'], db_social['Gender']))
print('DB v MG',cohen_kappa_score(db_social['Gender'], mg_social['Gender']))
print('SC v MG',cohen_kappa_score(sc_social['Gender'], mg_social['Gender']))
print('SC v DB',cohen_kappa_score(sc_social['Gender'], db_social['Gender']))
print('SC v AD',cohen_kappa_score(sc_social['Gender'], ad_social['Gender']))
print('SM v MG',cohen_kappa_score(sm_social['Gender'], mg_social['Gender']))
print('SM v DB',cohen_kappa_score(sm_social['Gender'], db_social['Gender']))
print('SM v AD',cohen_kappa_score(sm_social['Gender'], ad_social['Gender']))
print('SM v SC',cohen_kappa_score(sm_social['Gender'], sc_social['Gender']))

Kappa Scores Annotators:
AD v MG: 0.88
AD v DB: 0.8397435897435898
DB v MG 0.96
SC v MG 0.6
SC v DB 0.6365105008077545
SC v AD 0.6365105008077545
SM v MG 0.64
SM v DB 0.6784565916398714
SM v AD 0.5980707395498392
SM v SC 0.46457990115321257


In [23]:
print('Kappa Scores Gold:')
print('MG',cohen_kappa_score(social_gold_df['Gender'], mg_social['Gender']))
print('AD',cohen_kappa_score(social_gold_df['Gender'], ad_social['Gender']))
print('DB',cohen_kappa_score(social_gold_df['Gender'], db_social['Gender']))
print('SC',cohen_kappa_score(social_gold_df['Gender'], sc_social['Gender']))
print('SM',cohen_kappa_score(social_gold_df['Gender'], sm_social['Gender']))

Kappa Scores Gold:
MG 0.92
AD 0.8796147672552167
DB 0.9598715890850722
SC 0.6737357259380098
SM 0.6365105008077545


In [24]:
print('Accuracy Scores')
print('MG',accuracy_score(social_gold_df['Gender'], mg_social['Gender']))
print('AD',accuracy_score(social_gold_df['Gender'], ad_social['Gender']))
print('DB',accuracy_score(social_gold_df['Gender'], db_social['Gender']))
print('SC',accuracy_score(social_gold_df['Gender'], sc_social['Gender']))
print('SM',accuracy_score(social_gold_df['Gender'], sm_social['Gender']))

Accuracy Scores
MG 0.96
AD 0.94
DB 0.98
SC 0.84
SM 0.82


In [25]:
print('F1 minority class as positive')
print('MG', f1_score(social_gold_df['Gender'], mg_social['Gender'], pos_label='Male'))
print('AD', f1_score(social_gold_df['Gender'], ad_social['Gender'], pos_label='Male'))
print('DB', f1_score(social_gold_df['Gender'], db_social['Gender'], pos_label='Male'))
print('SC', f1_score(social_gold_df['Gender'], sc_social['Gender'], pos_label='Male'))
print('SM', f1_score(social_gold_df['Gender'], sm_social['Gender'], pos_label='Male'))

F1 minority class as positive
MG 0.9583333333333334
AD 0.9361702127659574
DB 0.9787234042553191
SC 0.8095238095238095
SM 0.8


In [26]:
fleiss_data = [('a1',i, mg_social.loc[i, 'Gender'] ) for i in range(len(mg_social))]
fleiss_data.extend([('a2',i, ad_social.loc[i, 'Gender'] ) for i in range(len(ad_social))])
fleiss_data.extend([('a3',i, db_social.loc[i, 'Gender'] ) for i in range(len(db_social))])
fleiss_data.extend([('a4',i, sc_social.loc[i, 'Gender'] ) for i in range(len(sc_social))])
fleiss_data.extend([('a5',i, sm_social.loc[i, 'Gender'] ) for i in range(len(sm_social))])
sr_task = agreement.AnnotationTask(data=fleiss_data)
print('Fleiss Kappa:',sr_task.multi_kappa())

Fleiss Kappa: 0.6941896024464829


#### Insurance

In [27]:
print('Kappa Scores Annotators:')
print('AD v MG:', cohen_kappa_score(ad_social['Insurance'], mg_social['Insurance']))
print('AD v DB:',cohen_kappa_score(ad_social['Insurance'], db_social['Insurance']))
print('DB v MG',cohen_kappa_score(db_social['Insurance'], mg_social['Insurance']))
print('SC v MG',cohen_kappa_score(sc_social['Insurance'], mg_social['Insurance']))
print('SC v DB',cohen_kappa_score(sc_social['Insurance'], db_social['Insurance']))
print('SC v AD',cohen_kappa_score(sc_social['Insurance'], ad_social['Insurance']))
print('SM v MG',cohen_kappa_score(sm_social['Insurance'], mg_social['Insurance']))
print('SM v DB',cohen_kappa_score(sm_social['Insurance'], db_social['Insurance']))
print('SM v AD',cohen_kappa_score(sm_social['Insurance'], ad_social['Insurance']))
print('SM v SC',cohen_kappa_score(sm_social['Insurance'], sc_social['Insurance']))

Kappa Scores Annotators:
AD v MG: 0.7030878859857482
AD v DB: 0.7297297297297298
DB v MG 0.6067415730337079
SC v MG 0.0
SC v DB 0.0
SC v AD 0.0
SM v MG 0.09584086799276681
SM v DB 0.01869158878504662
SM v AD 0.18738049713193128
SM v SC 0.0


In [28]:
print('Kappa Scores Gold:')
print('MG',cohen_kappa_score(social_gold_df['insurance'], mg_social['Insurance']))
print('AD',cohen_kappa_score(social_gold_df['insurance'], ad_social['Insurance']))
print('DB',cohen_kappa_score(social_gold_df['insurance'], db_social['Insurance']))
print('SC',cohen_kappa_score(social_gold_df['insurance'], sc_social['Insurance']))
print('SM',cohen_kappa_score(social_gold_df['insurance'], sm_social['Insurance']))

Kappa Scores Gold:
MG 0.20792079207920788
AD 0.39560439560439553
DB 0.42105263157894735
SC 0.0
SM -0.06194690265486713


In [29]:
print('Accuracy Scores')
print('MG',accuracy_score(social_gold_df['insurance'], mg_social['Insurance']))
print('AD',accuracy_score(social_gold_df['insurance'], ad_social['Insurance']))
print('DB',accuracy_score(social_gold_df['insurance'], db_social['Insurance']))
print('SC',accuracy_score(social_gold_df['insurance'], sc_social['Insurance']))
print('SM',accuracy_score(social_gold_df['insurance'], sm_social['Insurance']))

Accuracy Scores
MG 0.68
AD 0.78
DB 0.78
SC 0.7
SM 0.52


In [30]:
print('F1 minority class as positive')
print('MG', f1_score(social_gold_df['insurance'], mg_social['Insurance'], pos_label='low'))
print('AD', f1_score(social_gold_df['insurance'], ad_social['Insurance'], pos_label='low'))
print('DB', f1_score(social_gold_df['insurance'], db_social['Insurance'], pos_label='low'))
print('SC', f1_score(social_gold_df['insurance'], sc_social['Insurance'], pos_label='low'))
print('SM', f1_score(social_gold_df['insurance'], sm_social['Insurance'], pos_label='low'))

F1 minority class as positive
MG 0.42857142857142855
AD 0.5217391304347827
DB 0.56
SC 0.0
SM 0.29411764705882354


In [31]:
fleiss_data = [('a1',i, mg_social.loc[i, 'Insurance'] ) for i in range(len(mg_social))]
fleiss_data.extend([('a2',i, ad_social.loc[i, 'Insurance'] ) for i in range(len(ad_social))])
fleiss_data.extend([('a3',i, db_social.loc[i, 'Insurance'] ) for i in range(len(db_social))])
fleiss_data.extend([('a4',i, sc_social.loc[i, 'Insurance'] ) for i in range(len(sc_social))])
fleiss_data.extend([('a5',i, sm_social.loc[i, 'Insurance'] ) for i in range(len(sm_social))])
sr_task = agreement.AnnotationTask(data=fleiss_data)
print('Fleiss Kappa:',sr_task.multi_kappa())

Fleiss Kappa: 0.24334879179887736


## Nursing 

#### Race

In [32]:
print('Kappa Scores Annotators:')
print('AD v MG:', cohen_kappa_score(ad_nurse['Race'], mg_nurse['Race']))
print('AD v DB:',cohen_kappa_score(ad_nurse['Race'], db_nurse['Race']))
print('DB v MG',cohen_kappa_score(db_nurse['Race'], mg_nurse['Race']))
print('SC v MG',cohen_kappa_score(sc_nurse['Race'], mg_nurse['Race']))
print('SC v DB',cohen_kappa_score(sc_nurse['Race'], db_nurse['Race']))
print('SC v AD',cohen_kappa_score(sc_nurse['Race'], ad_nurse['Race']))
print('SM v MG',cohen_kappa_score(sm_nurse['Race'], mg_nurse['Race']))
print('SM v DB',cohen_kappa_score(sm_nurse['Race'], db_nurse['Race']))
print('SM v AD',cohen_kappa_score(sm_nurse['Race'], ad_nurse['Race']))
print('SM v SC',cohen_kappa_score(sm_nurse['Race'], sc_nurse['Race']))

Kappa Scores Annotators:
AD v MG: 0.6688741721854304
AD v DB: 0.6212121212121212
DB v MG 0.5033112582781457
SC v MG 0.5033112582781457
SC v DB 0.4318181818181819
SC v AD 0.6212121212121212
SM v MG 0.2857142857142857
SM v DB 0.02298850574712641
SM v AD 0.2528735632183907
SM v SC 0.13793103448275856


In [33]:
print('Kappa Scores Gold:')
print('MG',cohen_kappa_score(nurse_gold_df['Race_group'], mg_nurse['Race']))
print('AD',cohen_kappa_score(nurse_gold_df['Race_group'], ad_nurse['Race']))
print('DB',cohen_kappa_score(nurse_gold_df['Race_group'], db_nurse['Race']))
print('SC',cohen_kappa_score(nurse_gold_df['Race_group'], sc_nurse['Race']))
print('SM',cohen_kappa_score(nurse_gold_df['Race_group'], sm_nurse['Race']))

Kappa Scores Gold:
MG 0.4655581947743468
AD 0.5591939546599496
DB 0.3073047858942065
SC 0.3073047858942065
SM 0.30693069306930687


In [34]:
print('Accuracy Scores')
print('MG',accuracy_score(nurse_gold_df['Race_group'], mg_nurse['Race']))
print('AD',accuracy_score(nurse_gold_df['Race_group'], ad_nurse['Race']))
print('DB',accuracy_score(nurse_gold_df['Race_group'], db_nurse['Race']))
print('SC',accuracy_score(nurse_gold_df['Race_group'], sc_nurse['Race']))
print('SM',accuracy_score(nurse_gold_df['Race_group'], sm_nurse['Race']))

Accuracy Scores
MG 0.82
AD 0.86
DB 0.78
SC 0.78
SM 0.72


In [35]:
print('F1 minority class as positive')
print('MG', f1_score(nurse_gold_df['Race_group'], mg_nurse['Race'], pos_label='NonWhite'))
print('AD', f1_score(nurse_gold_df['Race_group'], ad_nurse['Race'], pos_label='NonWhite'))
print('DB', f1_score(nurse_gold_df['Race_group'], db_nurse['Race'], pos_label='NonWhite'))
print('SC', f1_score(nurse_gold_df['Race_group'], sc_nurse['Race'], pos_label='NonWhite'))
print('SM', f1_score(nurse_gold_df['Race_group'], sm_nurse['Race'], pos_label='NonWhite'))

F1 minority class as positive
MG 0.5714285714285714
AD 0.631578947368421
DB 0.42105263157894735
SC 0.42105263157894735
SM 0.5


In [36]:
fleiss_data = [('a1',i, mg_nurse.loc[i, 'Race'] ) for i in range(len(mg_nurse))]
fleiss_data.extend([('a2',i, ad_nurse.loc[i, 'Race'] ) for i in range(len(ad_nurse))])
fleiss_data.extend([('a3',i, db_nurse.loc[i, 'Race'] ) for i in range(len(db_nurse))])
fleiss_data.extend([('a4',i, sc_nurse.loc[i, 'Race'] ) for i in range(len(sc_nurse))])
fleiss_data.extend([('a5',i, sm_nurse.loc[i, 'Race'] ) for i in range(len(sm_nurse))])
sr_task = agreement.AnnotationTask(data=fleiss_data)
print('Fleiss Kappa:',sr_task.multi_kappa())

Fleiss Kappa: 0.3637941006362059


#### Gender

In [37]:
print('Kappa Scores Annotators:')
print('AD v MG:', cohen_kappa_score(ad_nurse['Gender'], mg_nurse['Gender']))
print('AD v DB:',cohen_kappa_score(ad_nurse['Gender'], db_nurse['Gender']))
print('DB v MG',cohen_kappa_score(db_nurse['Gender'], mg_nurse['Gender']))
print('SC v MG',cohen_kappa_score(sc_nurse['Gender'], mg_nurse['Gender']))
print('SC v DB',cohen_kappa_score(sc_nurse['Gender'], db_nurse['Gender']))
print('SC v AD',cohen_kappa_score(sc_nurse['Gender'], ad_nurse['Gender']))
print('SM v MG',cohen_kappa_score(sm_nurse['Gender'], mg_nurse['Gender']))
print('SM v DB',cohen_kappa_score(sm_nurse['Gender'], db_nurse['Gender']))
print('SM v AD',cohen_kappa_score(sm_nurse['Gender'], ad_nurse['Gender']))
print('SM v SC',cohen_kappa_score(sm_nurse['Gender'], sc_nurse['Gender']))

Kappa Scores Annotators:
AD v MG: 0.6815286624203822
AD v DB: 0.5860927152317881
DB v MG 0.6044303797468354
SC v MG 0.5528455284552846
SC v DB 0.316546762589928
SC v AD 0.4274809160305344
SM v MG 0.48330683624801274
SM v DB 0.5393634840871022
SM v AD 0.5513866231647635
SM v SC 0.3984962406015038


In [38]:
print('Kappa Scores Gold:')
print('MG',cohen_kappa_score(nurse_gold_df['Gender'], mg_nurse['Gender']))
print('AD',cohen_kappa_score(nurse_gold_df['Gender'], ad_nurse['Gender']))
print('DB',cohen_kappa_score(nurse_gold_df['Gender'], db_nurse['Gender']))
print('SC',cohen_kappa_score(nurse_gold_df['Gender'], sc_nurse['Gender']))
print('SM',cohen_kappa_score(nurse_gold_df['Gender'], sm_nurse['Gender']))

Kappa Scores Gold:
MG 0.5980707395498392
AD 0.6845425867507886
DB 0.5356037151702787
SC 0.5378151260504203
SM 0.4113029827315541


In [39]:
print('Accuracy Scores')
print('MG',accuracy_score(nurse_gold_df['Gender'], mg_nurse['Gender']))
print('AD',accuracy_score(nurse_gold_df['Gender'], ad_nurse['Gender']))
print('DB',accuracy_score(nurse_gold_df['Gender'], db_nurse['Gender']))
print('SC',accuracy_score(nurse_gold_df['Gender'], sc_nurse['Gender']))
print('SM',accuracy_score(nurse_gold_df['Gender'], sm_nurse['Gender']))

Accuracy Scores
MG 0.8
AD 0.84
DB 0.76
SC 0.78
SM 0.7


In [40]:
print('F1 minority class as positive')
print('MG', f1_score(nurse_gold_df['Gender'], mg_nurse['Gender'], pos_label='Male'))
print('AD', f1_score(nurse_gold_df['Gender'], ad_nurse['Gender'], pos_label='Male'))
print('DB', f1_score(nurse_gold_df['Gender'], db_nurse['Gender'], pos_label='Male'))
print('SC', f1_score(nurse_gold_df['Gender'], sc_nurse['Gender'], pos_label='Male'))
print('SM', f1_score(nurse_gold_df['Gender'], sm_nurse['Gender'], pos_label='Male'))

F1 minority class as positive
MG 0.7826086956521738
AD 0.84
DB 0.7777777777777778
SC 0.7027027027027029
SM 0.7058823529411765


In [41]:
fleiss_data = [('a1',i, mg_nurse.loc[i, 'Gender'] ) for i in range(len(mg_nurse))]
fleiss_data.extend([('a2',i, ad_nurse.loc[i, 'Gender'] ) for i in range(len(ad_nurse))])
fleiss_data.extend([('a3',i, db_nurse.loc[i, 'Gender'] ) for i in range(len(db_nurse))])
fleiss_data.extend([('a4',i, sc_nurse.loc[i, 'Gender'] ) for i in range(len(sc_nurse))])
fleiss_data.extend([('a5',i, sm_nurse.loc[i, 'Gender'] ) for i in range(len(sm_nurse))])
sr_task = agreement.AnnotationTask(data=fleiss_data)
print('Fleiss Kappa:',sr_task.multi_kappa())

Fleiss Kappa: 0.5105005526606663


#### Insurance

In [42]:
print('Kappa Scores Annotators:')
print('AD v MG:', cohen_kappa_score(ad_nurse['Insurance'], mg_nurse['Insurance']))
print('AD v DB:',cohen_kappa_score(ad_nurse['Insurance'], db_nurse['Insurance']))
print('DB v MG',cohen_kappa_score(db_nurse['Insurance'], mg_nurse['Insurance']))
print('SC v MG',cohen_kappa_score(sc_nurse['Insurance'], mg_nurse['Insurance']))
print('SC v DB',cohen_kappa_score(sc_nurse['Insurance'], db_nurse['Insurance']))
print('SC v AD',cohen_kappa_score(sc_nurse['Insurance'], ad_nurse['Insurance']))
print('SM v MG',cohen_kappa_score(sm_nurse['Insurance'], mg_nurse['Insurance']))
print('SM v DB',cohen_kappa_score(sm_nurse['Insurance'], db_nurse['Insurance']))
print('SM v AD',cohen_kappa_score(sm_nurse['Insurance'], ad_nurse['Insurance']))
print('SM v SC',cohen_kappa_score(sm_nurse['Insurance'], sc_nurse['Insurance']))

Kappa Scores Annotators:
AD v MG: 0.0
AD v DB: 0.48453608247422675
DB v MG 0.0
SC v MG nan
SC v DB 0.0
SC v AD 0.0
SM v MG 0.0
SM v DB -0.0387811634349029
SM v AD 0.02088772845953002
SM v SC 0.0


/home/marco/miniconda3/envs/bwh_models/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


In [43]:
print('Kappa Scores Gold:')
print('MG',cohen_kappa_score(nurse_gold_df['insurance'], mg_nurse['Insurance']))
print('AD',cohen_kappa_score(nurse_gold_df['insurance'], ad_nurse['Insurance']))
print('DB',cohen_kappa_score(nurse_gold_df['insurance'], db_nurse['Insurance']))
print('SC',cohen_kappa_score(nurse_gold_df['insurance'], sc_nurse['Insurance']))
print('SM',cohen_kappa_score(nurse_gold_df['insurance'], sm_nurse['Insurance']))

Kappa Scores Gold:
MG 0.0
AD 0.34497816593886466
DB 0.2227979274611399
SC 0.0
SM -0.1124121779859486


In [44]:
print('Accuracy Scores')
print('MG',accuracy_score(nurse_gold_df['insurance'], mg_nurse['Insurance']))
print('AD',accuracy_score(nurse_gold_df['insurance'], ad_nurse['Insurance']))
print('DB',accuracy_score(nurse_gold_df['insurance'], db_nurse['Insurance']))
print('SC',accuracy_score(nurse_gold_df['insurance'], sc_nurse['Insurance']))
print('SM',accuracy_score(nurse_gold_df['insurance'], sm_nurse['Insurance']))

Accuracy Scores
MG 0.86
AD 0.88
DB 0.88
SC 0.86
SM 0.62


In [45]:
print('F1 minority class as positive')
print('MG', f1_score(nurse_gold_df['insurance'], mg_nurse['Insurance'], pos_label='low'))
print('AD', f1_score(nurse_gold_df['insurance'], ad_nurse['Insurance'], pos_label='low'))
print('DB', f1_score(nurse_gold_df['insurance'], db_nurse['Insurance'], pos_label='low'))
print('SC', f1_score(nurse_gold_df['insurance'], sc_nurse['Insurance'], pos_label='low'))
print('SM', f1_score(nurse_gold_df['insurance'], sm_nurse['Insurance'], pos_label='low'))

F1 minority class as positive
MG 0.0
AD 0.4
DB 0.25
SC 0.0
SM 0.09523809523809523


In [46]:
fleiss_data = [('a1',i, mg_nurse.loc[i, 'Insurance'] ) for i in range(len(mg_nurse))]
fleiss_data.extend([('a2',i, ad_nurse.loc[i, 'Insurance'] ) for i in range(len(ad_nurse))])
fleiss_data.extend([('a3',i, db_nurse.loc[i, 'Insurance'] ) for i in range(len(db_nurse))])
fleiss_data.extend([('a4',i, sc_nurse.loc[i, 'Insurance'] ) for i in range(len(sc_nurse))])
fleiss_data.extend([('a5',i, sm_nurse.loc[i, 'Insurance'] ) for i in range(len(sm_nurse))])
sr_task = agreement.AnnotationTask(data=fleiss_data)
print('Fleiss Kappa:',sr_task.multi_kappa())

Fleiss Kappa: 0.023549684089604117


## Physician

#### Race

In [47]:
print('Kappa Scores Annotators:')
print('AD v MG:', cohen_kappa_score(ad_phys['Race'], mg_phys['Race']))
print('AD v DB:',cohen_kappa_score(ad_phys['Race'], db_phys['Race']))
print('DB v MG',cohen_kappa_score(db_phys['Race'], mg_phys['Race']))
print('SC v MG',cohen_kappa_score(sc_phys['Race'], mg_phys['Race']))
print('SC v DB',cohen_kappa_score(sc_phys['Race'], db_phys['Race']))
print('SC v AD',cohen_kappa_score(sc_phys['Race'], ad_phys['Race']))
print('SM v MG',cohen_kappa_score(sm_phys['Race'], mg_phys['Race']))
print('SM v DB',cohen_kappa_score(sm_phys['Race'], db_phys['Race']))
print('SM v AD',cohen_kappa_score(sm_phys['Race'], ad_phys['Race']))
print('SM v SC',cohen_kappa_score(sm_phys['Race'], sc_phys['Race']))

Kappa Scores Annotators:
AD v MG: 0.5978552278820376
AD v DB: 0.33734939759036153
DB v MG 0.5070422535211268
SC v MG 0.4339622641509434
SC v DB 0.3076923076923077
SC v AD 0.5652173913043479
SM v MG 0.2723492723492723
SM v DB 0.15841584158415833
SM v AD 0.12280701754385959
SM v SC 0.13978494623655924


In [48]:
print('Kappa Scores Gold:')
print('MG',cohen_kappa_score(phys_gold_df['Race_group'], mg_phys['Race']))
print('AD',cohen_kappa_score(phys_gold_df['Race_group'], ad_phys['Race']))
print('DB',cohen_kappa_score(phys_gold_df['Race_group'], db_phys['Race']))
print('SC',cohen_kappa_score(phys_gold_df['Race_group'], sc_phys['Race']))
print('SM',cohen_kappa_score(phys_gold_df['Race_group'], sm_phys['Race']))

Kappa Scores Gold:
MG 0.42455242966751916
AD 0.37923250564334077
DB 0.41860465116279066
SC 0.24657534246575352
SM -0.007677543186180413


In [49]:
print('Accuracy Scores')
print('MG',accuracy_score(phys_gold_df['Race_group'], mg_phys['Race']))
print('AD',accuracy_score(phys_gold_df['Race_group'], ad_phys['Race']))
print('DB',accuracy_score(phys_gold_df['Race_group'], db_phys['Race']))
print('SC',accuracy_score(phys_gold_df['Race_group'], sc_phys['Race']))
print('SM',accuracy_score(phys_gold_df['Race_group'], sm_phys['Race']))

Accuracy Scores
MG 0.82
AD 0.78
DB 0.8
SC 0.78
SM 0.58


In [50]:
print('F1 minority class as positive')
print('MG', f1_score(phys_gold_df['Race_group'], mg_phys['Race'], pos_label='NonWhite'))
print('AD', f1_score(phys_gold_df['Race_group'], ad_phys['Race'], pos_label='NonWhite'))
print('DB', f1_score(phys_gold_df['Race_group'], db_phys['Race'], pos_label='NonWhite'))
print('SC', f1_score(phys_gold_df['Race_group'], sc_phys['Race'], pos_label='NonWhite'))
print('SM', f1_score(phys_gold_df['Race_group'], sm_phys['Race'], pos_label='NonWhite'))

F1 minority class as positive
MG 0.5263157894736842
AD 0.5217391304347826
DB 0.5454545454545454
SC 0.35294117647058826
SM 0.27586206896551724


In [51]:
fleiss_data = [('a1',i, mg_phys.loc[i, 'Race'] ) for i in range(len(mg_phys))]
fleiss_data.extend([('a2',i, ad_phys.loc[i, 'Race'] ) for i in range(len(ad_phys))])
fleiss_data.extend([('a3',i, db_phys.loc[i, 'Race'] ) for i in range(len(db_phys))])
fleiss_data.extend([('a4',i, sc_phys.loc[i, 'Race'] ) for i in range(len(sc_phys))])
fleiss_data.extend([('a5',i, sm_phys.loc[i, 'Race'] ) for i in range(len(sm_phys))])
sr_task = agreement.AnnotationTask(data=fleiss_data)
print('Fleiss Kappa:',sr_task.multi_kappa())

Fleiss Kappa: 0.31964374072241447


#### Gender

In [52]:
print('Kappa Scores Annotators:')
print('AD v MG:', cohen_kappa_score(ad_phys['Gender'], mg_phys['Gender']))
print('AD v DB:',cohen_kappa_score(ad_phys['Gender'], db_phys['Gender']))
print('DB v MG',cohen_kappa_score(db_phys['Gender'], mg_phys['Gender']))
print('SC v MG',cohen_kappa_score(sc_phys['Gender'], mg_phys['Gender']))
print('SC v DB',cohen_kappa_score(sc_phys['Gender'], db_phys['Gender']))
print('SC v AD',cohen_kappa_score(sc_phys['Gender'], ad_phys['Gender']))
print('SM v MG',cohen_kappa_score(sm_phys['Gender'], mg_phys['Gender']))
print('SM v DB',cohen_kappa_score(sm_phys['Gender'], db_phys['Gender']))
print('SM v AD',cohen_kappa_score(sm_phys['Gender'], ad_phys['Gender']))
print('SM v SC',cohen_kappa_score(sm_phys['Gender'], sc_phys['Gender']))

Kappa Scores Annotators:
AD v MG: 0.9180327868852459
AD v DB: 0.9586776859504132
DB v MG 0.9592169657422512
SC v MG 0.6218487394957983
SC v DB 0.6581196581196581
SC v AD 0.6956521739130435
SM v MG 0.8360655737704918
SM v DB 0.8760330578512396
SM v AD 0.9166666666666666
SM v SC 0.6956521739130435


In [53]:
print('Kappa Scores Gold:')
print('MG',cohen_kappa_score(phys_gold_df['Gender'], mg_phys['Gender']))
print('AD',cohen_kappa_score(phys_gold_df['Gender'], ad_phys['Gender']))
print('DB',cohen_kappa_score(phys_gold_df['Gender'], db_phys['Gender']))
print('SC',cohen_kappa_score(phys_gold_df['Gender'], sc_phys['Gender']))
print('SM',cohen_kappa_score(phys_gold_df['Gender'], sm_phys['Gender']))

Kappa Scores Gold:
MG 0.9592169657422512
AD 0.9586776859504132
DB 0.9178981937602627
SC 0.6581196581196581
SM 0.8760330578512396


In [54]:
print('Accuracy Scores')
print('MG',accuracy_score(phys_gold_df['Gender'], mg_phys['Gender']))
print('AD',accuracy_score(phys_gold_df['Gender'], ad_phys['Gender']))
print('DB',accuracy_score(phys_gold_df['Gender'], db_phys['Gender']))
print('SC',accuracy_score(phys_gold_df['Gender'], sc_phys['Gender']))
print('SM',accuracy_score(phys_gold_df['Gender'], sm_phys['Gender']))

Accuracy Scores
MG 0.98
AD 0.98
DB 0.96
SC 0.84
SM 0.94


In [55]:
print('F1 minority class as positive')
print('MG', f1_score(phys_gold_df['Gender'], mg_phys['Gender'], pos_label='Male'))
print('AD', f1_score(phys_gold_df['Gender'], ad_phys['Gender'], pos_label='Male'))
print('DB', f1_score(phys_gold_df['Gender'], db_phys['Gender'], pos_label='Male'))
print('SC', f1_score(phys_gold_df['Gender'], sc_phys['Gender'], pos_label='Male'))
print('SM', f1_score(phys_gold_df['Gender'], sm_phys['Gender'], pos_label='Male'))

F1 minority class as positive
MG 0.9767441860465117
AD 0.975609756097561
DB 0.9523809523809523
SC 0.7777777777777778
SM 0.9268292682926829


In [56]:
fleiss_data = [('a1',i, mg_phys.loc[i, 'Gender'] ) for i in range(len(mg_phys))]
fleiss_data.extend([('a2',i, ad_phys.loc[i, 'Gender'] ) for i in range(len(ad_phys))])
fleiss_data.extend([('a3',i, db_phys.loc[i, 'Gender'] ) for i in range(len(db_phys))])
fleiss_data.extend([('a4',i, sc_phys.loc[i, 'Gender'] ) for i in range(len(sc_phys))])
fleiss_data.extend([('a5',i, sm_phys.loc[i, 'Gender'] ) for i in range(len(sm_phys))])
sr_task = agreement.AnnotationTask(data=fleiss_data)
print('Fleiss Kappa:',sr_task.multi_kappa())

Fleiss Kappa: 0.8158379373848985


#### Insurance

In [57]:
print('Kappa Scores Annotators:')
print('AD v MG:', cohen_kappa_score(ad_phys['Insurance'], mg_phys['Insurance']))
print('AD v DB:',cohen_kappa_score(ad_phys['Insurance'], db_phys['Insurance']))
print('DB v MG',cohen_kappa_score(db_phys['Insurance'], mg_phys['Insurance']))
print('SC v MG',cohen_kappa_score(sc_phys['Insurance'], mg_phys['Insurance']))
print('SC v DB',cohen_kappa_score(sc_phys['Insurance'], db_phys['Insurance']))
print('SC v AD',cohen_kappa_score(sc_phys['Insurance'], ad_phys['Insurance']))
print('SM v MG',cohen_kappa_score(sm_phys['Insurance'], mg_phys['Insurance']))
print('SM v DB',cohen_kappa_score(sm_phys['Insurance'], db_phys['Insurance']))
print('SM v AD',cohen_kappa_score(sm_phys['Insurance'], ad_phys['Insurance']))
print('SM v SC',cohen_kappa_score(sm_phys['Insurance'], sc_phys['Insurance']))

Kappa Scores Annotators:
AD v MG: 0.0
AD v DB: -0.050420168067226934
DB v MG 0.0
SC v MG nan
SC v DB 0.0
SC v AD 0.0
SM v MG 0.0
SM v DB 0.07975460122699385
SM v AD -0.10619469026548689
SM v SC 0.0


/home/marco/miniconda3/envs/bwh_models/lib/python3.9/site-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


In [58]:
print('Kappa Scores Gold:')
print('MG',cohen_kappa_score(phys_gold_df['insurance'], mg_phys['Insurance']))
print('AD',cohen_kappa_score(phys_gold_df['insurance'], ad_phys['Insurance']))
print('DB',cohen_kappa_score(phys_gold_df['insurance'], db_phys['Insurance']))
print('SC',cohen_kappa_score(phys_gold_df['insurance'], sc_phys['Insurance']))
print('SM',cohen_kappa_score(phys_gold_df['insurance'], sm_phys['Insurance']))

Kappa Scores Gold:
MG 0.0
AD 0.12177985948477743
DB 0.04306220095693769
SC 0.0
SM 0.21259842519685046


In [59]:
print('Accuracy Scores')
print('MG',accuracy_score(phys_gold_df['insurance'], mg_phys['Insurance']))
print('AD',accuracy_score(phys_gold_df['insurance'], ad_phys['Insurance']))
print('DB',accuracy_score(phys_gold_df['insurance'], db_phys['Insurance']))
print('SC',accuracy_score(phys_gold_df['insurance'], sc_phys['Insurance']))
print('SM',accuracy_score(phys_gold_df['insurance'], sm_phys['Insurance']))

Accuracy Scores
MG 0.68
AD 0.7
DB 0.68
SC 0.68
SM 0.68


In [60]:
print('F1 minority class as positive')
print('MG', f1_score(phys_gold_df['insurance'], mg_phys['Insurance'], pos_label='low'))
print('AD', f1_score(phys_gold_df['insurance'], ad_phys['Insurance'], pos_label='low'))
print('DB', f1_score(phys_gold_df['insurance'], db_phys['Insurance'], pos_label='low'))
print('SC', f1_score(phys_gold_df['insurance'], sc_phys['Insurance'], pos_label='low'))
print('SM', f1_score(phys_gold_df['insurance'], sm_phys['Insurance'], pos_label='low'))

F1 minority class as positive
MG 0.0
AD 0.21052631578947367
DB 0.1111111111111111
SC 0.0
SM 0.42857142857142855


In [61]:
fleiss_data = [('a1',i, mg_phys.loc[i, 'Insurance'] ) for i in range(len(mg_phys))]
fleiss_data.extend([('a2',i, ad_phys.loc[i, 'Insurance'] ) for i in range(len(ad_phys))])
fleiss_data.extend([('a3',i, db_phys.loc[i, 'Insurance'] ) for i in range(len(db_phys))])
fleiss_data.extend([('a4',i, sc_phys.loc[i, 'Insurance'] ) for i in range(len(sc_phys))])
fleiss_data.extend([('a5',i, sm_phys.loc[i, 'Insurance'] ) for i in range(len(sm_phys))])
sr_task = agreement.AnnotationTask(data=fleiss_data)
print('Fleiss Kappa:',sr_task.multi_kappa())

Fleiss Kappa: -0.009791921664627137


## Overall

In [62]:
mg_combined = pd.concat([mg_social, mg_nurse, mg_phys])
ad_combined = pd.concat([ad_social, ad_nurse, ad_phys])
db_combined = pd.concat([db_social, db_nurse, db_phys])
sc_combined = pd.concat([sc_social, sc_nurse, sc_phys])
sm_combined = pd.concat([sm_social, sm_nurse, sm_phys])

In [63]:
combined_gold_df = pd.concat([social_gold_df, nurse_gold_df, phys_gold_df])

In [64]:
mg_combined.reset_index(inplace=True, drop=True)
ad_combined.reset_index(inplace=True, drop=True)
db_combined.reset_index(inplace=True, drop=True)
sc_combined.reset_index(inplace=True, drop=True)
sm_combined.reset_index(inplace=True, drop=True)
combined_gold_df.reset_index(inplace=True, drop=True)

#### Race

In [65]:
print('Kappa Scores Annotators:')
print('AD v MG:', cohen_kappa_score(ad_combined['Race'], mg_combined['Race']))
print('AD v DB:',cohen_kappa_score(ad_combined['Race'], db_combined['Race']))
print('DB v MG',cohen_kappa_score(db_combined['Race'], mg_combined['Race']))
print('SC v MG',cohen_kappa_score(sc_combined['Race'], mg_combined['Race']))
print('SC v DB',cohen_kappa_score(sc_combined['Race'], db_combined['Race']))
print('SC v AD',cohen_kappa_score(sc_combined['Race'], ad_combined['Race']))
print('SM v MG',cohen_kappa_score(sm_combined['Race'], mg_combined['Race']))
print('SM v DB',cohen_kappa_score(sm_combined['Race'], db_combined['Race']))
print('SM v AD',cohen_kappa_score(sm_combined['Race'], ad_combined['Race']))
print('SM v SC',cohen_kappa_score(sm_combined['Race'], sc_combined['Race']))

Kappa Scores Annotators:
AD v MG: 0.6463376296762025
AD v DB: 0.5608899297423888
DB v MG 0.5520276642565232
SC v MG 0.3918171765573166
SC v DB 0.27330437687938525
SC v AD 0.47377213498162374
SM v MG 0.2804847260792729
SM v DB 0.18102402329531675
SM v AD 0.18102402329531675
SM v SC 0.1390555700495696


In [66]:
print('Kappa Scores Gold:')
print('MG',cohen_kappa_score(combined_gold_df['Race_group'], mg_combined['Race']))
print('AD',cohen_kappa_score(combined_gold_df['Race_group'], ad_combined['Race']))
print('DB',cohen_kappa_score(combined_gold_df['Race_group'], db_combined['Race']))
print('SC',cohen_kappa_score(combined_gold_df['Race_group'], sc_combined['Race']))
print('SM',cohen_kappa_score(combined_gold_df['Race_group'], sm_combined['Race']))

Kappa Scores Gold:
MG 0.3833927103315977
AD 0.39437353477468096
DB 0.39437353477468096
SC 0.14212181870174434
SM 0.2175096393740077


In [67]:
print('Accuracy Scores')
print('MG',accuracy_score(combined_gold_df['Race_group'], mg_combined['Race']))
print('AD',accuracy_score(combined_gold_df['Race_group'], ad_combined['Race']))
print('DB',accuracy_score(combined_gold_df['Race_group'], db_combined['Race']))
print('SC',accuracy_score(combined_gold_df['Race_group'], sc_combined['Race']))
print('SM',accuracy_score(combined_gold_df['Race_group'], sm_combined['Race']))

Accuracy Scores
MG 0.8
AD 0.7933333333333333
DB 0.7933333333333333
SC 0.7333333333333333
SM 0.6933333333333334


In [68]:
print('F1 minority class as positive')
print('MG', f1_score(combined_gold_df['Race_group'], mg_combined['Race'], pos_label='NonWhite'))
print('AD', f1_score(combined_gold_df['Race_group'], ad_combined['Race'], pos_label='NonWhite'))
print('DB', f1_score(combined_gold_df['Race_group'], db_combined['Race'], pos_label='NonWhite'))
print('SC', f1_score(combined_gold_df['Race_group'], sc_combined['Race'], pos_label='NonWhite'))
print('SM', f1_score(combined_gold_df['Race_group'], sm_combined['Race'], pos_label='NonWhite'))

F1 minority class as positive
MG 0.5
AD 0.5230769230769231
DB 0.5230769230769231
SC 0.28571428571428575
SM 0.425


In [69]:
fleiss_data = [('a1',i, mg_combined.loc[i, 'Race'] ) for i in range(len(mg_combined))]
fleiss_data.extend([('a2',i, ad_combined.loc[i, 'Race'] ) for i in range(len(ad_combined))])
fleiss_data.extend([('a3',i, db_combined.loc[i, 'Race'] ) for i in range(len(db_combined))])
fleiss_data.extend([('a4',i, sc_combined.loc[i, 'Race'] ) for i in range(len(sc_combined))])
fleiss_data.extend([('a5',i, sm_combined.loc[i, 'Race'] ) for i in range(len(sm_combined))])
sr_task = agreement.AnnotationTask(data=fleiss_data)
print('Fleiss Kappa:',sr_task.multi_kappa())

Fleiss Kappa: 0.3524179294758498


#### Gender

In [70]:
print('Kappa Scores Annotators:')
print('AD v MG:', cohen_kappa_score(ad_combined['Gender'], mg_combined['Gender']))
print('AD v DB:',cohen_kappa_score(ad_combined['Gender'], db_combined['Gender']))
print('DB v MG',cohen_kappa_score(db_combined['Gender'], mg_combined['Gender']))
print('SC v MG',cohen_kappa_score(sc_combined['Gender'], mg_combined['Gender']))
print('SC v DB',cohen_kappa_score(sc_combined['Gender'], db_combined['Gender']))
print('SC v AD',cohen_kappa_score(sc_combined['Gender'], ad_combined['Gender']))
print('SM v MG',cohen_kappa_score(sm_combined['Gender'], mg_combined['Gender']))
print('SM v DB',cohen_kappa_score(sm_combined['Gender'], db_combined['Gender']))
print('SM v AD',cohen_kappa_score(sm_combined['Gender'], ad_combined['Gender']))
print('SM v SC',cohen_kappa_score(sm_combined['Gender'], sc_combined['Gender']))

Kappa Scores Annotators:
AD v MG: 0.826296098343132
AD v DB: 0.8002131060202451
DB v MG 0.8402272323806143
SC v MG 0.5924651331280565
SC v DB 0.5243966883917561
SC v AD 0.5808545159545699
SM v MG 0.6523444464253878
SM v DB 0.7070832593644594
SM v AD 0.6926777124532335
SM v SC 0.5109581597536678


In [71]:
print('Kappa Scores Gold:')
print('MG',cohen_kappa_score(combined_gold_df['Gender'], mg_combined['Gender']))
print('AD',cohen_kappa_score(combined_gold_df['Gender'], ad_combined['Gender']))
print('DB',cohen_kappa_score(combined_gold_df['Gender'], db_combined['Gender']))
print('SC',cohen_kappa_score(combined_gold_df['Gender'], sc_combined['Gender']))
print('SM',cohen_kappa_score(combined_gold_df['Gender'], sm_combined['Gender']))

Kappa Scores Gold:
MG 0.8255501878690285
AD 0.8392282958199357
DB 0.8006379585326954
SC 0.6243739565943238
SM 0.6376811594202898


In [72]:
print('Accuracy Scores')
print('MG',accuracy_score(combined_gold_df['Gender'], mg_combined['Gender']))
print('AD',accuracy_score(combined_gold_df['Gender'], ad_combined['Gender']))
print('DB',accuracy_score(combined_gold_df['Gender'], db_combined['Gender']))
print('SC',accuracy_score(combined_gold_df['Gender'], sc_combined['Gender']))
print('SM',accuracy_score(combined_gold_df['Gender'], sm_combined['Gender']))

Accuracy Scores
MG 0.9133333333333333
AD 0.92
DB 0.9
SC 0.82
SM 0.82


In [73]:
print('F1 minority class as positive')
print('MG', f1_score(combined_gold_df['Gender'], mg_combined['Gender'], pos_label='Male'))
print('AD', f1_score(combined_gold_df['Gender'], ad_combined['Gender'], pos_label='Male'))
print('DB', f1_score(combined_gold_df['Gender'], db_combined['Gender'], pos_label='Male'))
print('SC', f1_score(combined_gold_df['Gender'], sc_combined['Gender'], pos_label='Male'))
print('SM', f1_score(combined_gold_df['Gender'], sm_combined['Gender'], pos_label='Male'))

F1 minority class as positive
MG 0.9051094890510948
AD 0.9130434782608695
DB 0.8951048951048951
SC 0.7652173913043478
SM 0.8029197080291971


In [74]:
fleiss_data = [('a1',i, mg_combined.loc[i, 'Gender'] ) for i in range(len(mg_combined))]
fleiss_data.extend([('a2',i, ad_combined.loc[i, 'Gender'] ) for i in range(len(ad_combined))])
fleiss_data.extend([('a3',i, db_combined.loc[i, 'Gender'] ) for i in range(len(db_combined))])
fleiss_data.extend([('a4',i, sc_combined.loc[i, 'Gender'] ) for i in range(len(sc_combined))])
fleiss_data.extend([('a5',i, sm_combined.loc[i, 'Gender'] ) for i in range(len(sm_combined))])
sr_task = agreement.AnnotationTask(data=fleiss_data)
print('Fleiss Kappa:',sr_task.multi_kappa())

Fleiss Kappa: 0.6732026143790848


#### Insurance

In [75]:
print('Kappa Scores Annotators:')
print('AD v MG:', cohen_kappa_score(ad_combined['Insurance'], mg_combined['Insurance']))
print('AD v DB:',cohen_kappa_score(ad_combined['Insurance'], db_combined['Insurance']))
print('DB v MG',cohen_kappa_score(db_combined['Insurance'], mg_combined['Insurance']))
print('SC v MG',cohen_kappa_score(sc_combined['Insurance'], mg_combined['Insurance']))
print('SC v DB',cohen_kappa_score(sc_combined['Insurance'], db_combined['Insurance']))
print('SC v AD',cohen_kappa_score(sc_combined['Insurance'], ad_combined['Insurance']))
print('SM v MG',cohen_kappa_score(sm_combined['Insurance'], mg_combined['Insurance']))
print('SM v DB',cohen_kappa_score(sm_combined['Insurance'], db_combined['Insurance']))
print('SM v AD',cohen_kappa_score(sm_combined['Insurance'], ad_combined['Insurance']))
print('SM v SC',cohen_kappa_score(sm_combined['Insurance'], sc_combined['Insurance']))

Kappa Scores Annotators:
AD v MG: 0.5523602821486706
AD v DB: 0.5523602821486706
DB v MG 0.5788882650196518
SC v MG 0.0
SC v DB 0.0
SC v AD 0.0
SM v MG 0.08366533864541836
SM v DB 0.043824701195219196
SM v AD 0.07114624505928846
SM v SC 0.0


In [76]:
print('Kappa Scores Gold:')
print('MG',cohen_kappa_score(combined_gold_df['insurance'], mg_combined['Insurance']))
print('AD',cohen_kappa_score(combined_gold_df['insurance'], ad_combined['Insurance']))
print('DB',cohen_kappa_score(combined_gold_df['insurance'], db_combined['Insurance']))
print('SC',cohen_kappa_score(combined_gold_df['insurance'], sc_combined['Insurance']))
print('SM',cohen_kappa_score(combined_gold_df['insurance'], sm_combined['Insurance']))

Kappa Scores Gold:
MG 0.12188531972380656
AD 0.2874109263657957
DB 0.256979885920144
SC 0.0
SM 0.019933554817275767


In [77]:
print('Accuracy Scores')
print('MG',accuracy_score(combined_gold_df['insurance'], mg_combined['Insurance']))
print('AD',accuracy_score(combined_gold_df['insurance'], ad_combined['Insurance']))
print('DB',accuracy_score(combined_gold_df['insurance'], db_combined['Insurance']))
print('SC',accuracy_score(combined_gold_df['insurance'], sc_combined['Insurance']))
print('SM',accuracy_score(combined_gold_df['insurance'], sm_combined['Insurance']))

Accuracy Scores
MG 0.74
AD 0.7866666666666666
DB 0.78
SC 0.7466666666666667
SM 0.6066666666666667


In [78]:
print('F1 minority class as positive')
print('MG', f1_score(combined_gold_df['insurance'], mg_combined['Insurance'], pos_label='low'))
print('AD', f1_score(combined_gold_df['insurance'], ad_combined['Insurance'], pos_label='low'))
print('DB', f1_score(combined_gold_df['insurance'], db_combined['Insurance'], pos_label='low'))
print('SC', f1_score(combined_gold_df['insurance'], sc_combined['Insurance'], pos_label='low'))
print('SM', f1_score(combined_gold_df['insurance'], sm_combined['Insurance'], pos_label='low'))

F1 minority class as positive
MG 0.23529411764705882
AD 0.3846153846153846
DB 0.35294117647058826
SC 0.0
SM 0.2891566265060241


In [79]:
fleiss_data = [('a1',i, mg_combined.loc[i, 'Insurance'] ) for i in range(len(mg_combined))]
fleiss_data.extend([('a2',i, ad_combined.loc[i, 'Insurance'] ) for i in range(len(ad_combined))])
fleiss_data.extend([('a3',i, db_combined.loc[i, 'Insurance'] ) for i in range(len(db_combined))])
fleiss_data.extend([('a4',i, sc_combined.loc[i, 'Insurance'] ) for i in range(len(sc_combined))])
fleiss_data.extend([('a5',i, sm_combined.loc[i, 'Insurance'] ) for i in range(len(sm_combined))])
sr_task = agreement.AnnotationTask(data=fleiss_data)
print('Fleiss Kappa:',sr_task.multi_kappa())

Fleiss Kappa: 0.16476021927742046
